<a href="https://colab.research.google.com/github/2359181042/GNN_cs224w/blob/main/Wells_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6 MB 2.0 MB/s 
     |████████████████████████████████| 1.4 MB 2.0 MB/s 
     |████████████████████████████████| 222 kB 5.4 MB/s 
     |████████████████████████████████| 376 kB 44.7 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import os
import json
import numpy as np
import pandas as pd
DATA_PATH = 'gdrive/My Drive/DaguRiver/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')